# Libraries

In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn import metrics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [4]:
%run Data_preprocessing.ipynb

************************************************************
There are 6 set of X
X_*_nozero
------------------------------------------------------------
Normalized version
X_*_norm
------------------------------------------------------------
There are 4 set of Y
y_train_t1, y_train_t2, y_valid_t1, y_valid_t2
when training, please use: 'y_train_t1_value,y_train_t2_value,y_valid_t1_value,y_valid_t2_value'
************************************************************


In [5]:
print(X_train_norm.shape)
print(X_valid_norm.shape)
print(X_test_norm.shape)

(16760, 61)
(2394, 61)
(4790, 61)


# Imbalanced Learning

In [6]:
Full = pd.DataFrame(np.concatenate((X_train_norm,pd.DataFrame(y_train_t1.iloc[:,-1])),axis=1))
Full

label0 = Full[Full[61]==0]
label1 = Full[Full[61]==1]
print(label0.shape,label1.shape)

(15535, 62) (1225, 62)


In [7]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler()
x_over, y_over = oversample.fit_resample(X_train_norm, y_train_t1.iloc[:,-1])


y_over = pd.DataFrame(y_over)
y_over.shape
x_over.shape

(31070, 61)

In [8]:
Full = pd.DataFrame(np.concatenate((x_over,y_over),axis=1))
Full

label0 = Full[Full[61]==0]
label1 = Full[Full[61]==1]
print(label0.shape,label1.shape)

(15535, 62) (15535, 62)


In [9]:
X_train_norm=x_over

y_train_t1 = y_over

In [10]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver="lbfgs",
                    alpha=1e-5,
                    hidden_layer_sizes=(5,2),
                    random_state=1
                   )

clf.fit(X_train_norm,y_train_t1)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

# Model Accuracy

In [11]:
num_correct = 0
num_samples = 0

# Set model to eval
model.eval()

with torch.no_grad():
    for x, y in train_loader:
        x = x.to(device=device)
        y.to(device=device)
        #y = torch.squeeze(y.to(device=device), 1)
        #print(y.shape)

        scores = model(x)
        predictions = scores
        predictions = predictions.apply_( lambda x: 1 if x >= 0.5 else 0 )
        num_correct += int((predictions == y).sum())
        num_samples += predictions.size(0)

# Toggle model back to train
model.train()
#print(num_correct , num_samples)
num_correct / num_samples

NameError: name 'model' is not defined

In [ ]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in train_loader:
            x = x.to(device=device)
            y.to(device=device)
            #y = torch.squeeze(y.to(device=device), 1)
            #print(y.shape)

            scores = model(x)
            predictions = scores
            predictions = predictions.apply_( lambda x: 1 if x >= 0.5 else 0 )
            num_correct += int((predictions == y).sum())
            num_samples += predictions.size(0)

    # Toggle model back to train
    model.train()
    return num_correct / num_samples

In [ ]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f} %")
#print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f} %")

# Model Performance on validation dataset

In [ ]:
X2 = X_tensor(X_valid_norm)
y2 = y_tensor(y_valid_t1.iloc[:,-1])

Valid_datasets = CusDatasetLoader(X2, y2)
Valid_loader = DataLoader(dataset=Valid_datasets, batch_size=batch_size, shuffle=True)
print(f"Accuracy on valid set: {check_accuracy(Valid_loader, model)*100:.2f} %")

In [ ]:
y_true = np.array(y_valid_t1.iloc[:,[-1]])
all_y_pred = []

model.eval()
with torch.no_grad():
    for x, y in Valid_loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #y = torch.squeeze(y.to(device=device), 1)
            
            pred_y1 = torch.sigmoid(model(x))
            y_pred = pred_y1.squeeze(-1).detach().numpy()
            all_y_pred = np.append(all_y_pred, y_pred)

print(y_true.shape)
print(all_y_pred.reshape(-1,1).shape)
fpr, tpr, _ = metrics.roc_curve(y_true, all_y_pred)
roc_auc = metrics.roc_auc_score(y_true, all_y_pred)

model.train()

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k-')
plt.plot(fpr, tpr, label='FN(area = {:.3f})'.format(roc_auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title("ROC curve")
plt.legend(loc="best")
plt.show()

# Test

In [ ]:
def test(loader, model):
    # Set model to eval
    model.eval()
    res = torch.tensor([], dtype=torch.int64)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y.to(device=device)

            scores = model(x)
            predictions = scores
            res =  torch.cat((predictions, res), 0)  
    model.train()
   
    return res

In [ ]:
X_test_mean
x_valid_mean=X_test_mean.values
x_valid_mean_scaled = scaler.fit_transform(x_valid_mean)
X_test_norm=pd.DataFrame(x_valid_mean_scaled)
X_test_norm

In [ ]:
X_test_final = X_tensor(X_test_norm)
y_final = torch.zeros([4790,1])

test_datasets = CusDatasetLoader(X_test_final, y_final)
test_loader = DataLoader(dataset=test_datasets, batch_size=batch_size, shuffle=True)

result = test(test_loader, model)
result

In [ ]:

#df.index = ['Row_1', 'Row_2', 'Row_3', 'Row_4']
res1 = pd.DataFrame(result.tolist(), columns=["Predicted"])
res1.index = X_test.index
res1.index.name = 'Id'
pd.DataFrame(res1).to_csv('out.csv')
#print(torch.count_nonzero(torch.from_numpy(np.array(res1)).to(torch.float32)))
res1